In [1]:
from sklearn.preprocessing import StandardScaler # scales variables to be mean=0, sd=1
from sklearn.pipeline import Pipeline # in case multiple things are performed at the same time
from sklearn.ensemble import GradientBoostingRegressor # Model 3
from sklearn.ensemble import RandomForestRegressor # Model 2
from sklearn.linear_model import LassoCV # Model 1
from sklearn.linear_model import Lasso # To save time and not do the Cross-Validation every time
from sklearn.svm import LinearSVR # Model 4
from sklearn.model_selection import GridSearchCV # Hypertuning of parameters
from sklearn.ensemble import VotingRegressor # Ensemble estimator
import pandas as pd
import numpy as np
import time

In [2]:
def CombineAttributes(data, var_list):
    
    for i in var_list:
        for j in var_list:
            
            if i == j:
                name = str(i)+ '_square'
                data[name] = data.loc[:, i] * data.loc[:, i]

            elif i <=j:
                name =  str(i)+ '_' +str(j)
                data[name] = data.loc[:, i] * data.loc[:, j]
            else:
                pass

    return data

In [349]:
dataframe = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\bayern_complete_1.xlsx')
dataframe.head()

,county_id,year,asyl,city_id,city,cost_o_liv,BP_votes,CSU_votes,SPD_votes,ödp_votes,...,total_unemployed,unemployed15_20,unemployed15_25,unemployed55_65,unemployed_foreigners,unemployed_handycapped,unemployed_longtime,votes,youth_list,ödp
0,9162,2020,5166.0,9162,München,2861.34,3779,216444,116764,6073.0,...,32465.0,455.0,1957.0,6480.0,14411.0,2386.0,8227.0,NaN,0,0
1,9162,2020,5166.0,9162,München,2861.34,3779,216444,116764,6073.0,...,32465.0,455.0,1957.0,6480.0,14411.0,2386.0,8227.0,NaN,0,0
2,9162,2020,5166.0,9162,München,2861.34,3779,216444,116764,6073.0,...,32465.0,455.0,1957.0,6480.0,14411.0,2386.0,8227.0,NaN,0,0
3,9162,2020,5166.0,9162,München,2861.34,3779,216444,116764,6073.0,...,32465.0,455.0,1957.0,6480.0,14411.0,2386.0,8227.0,NaN,0,0
4,9162,2020,5166.0,9162,München,2861.34,3779,216444,116764,6073.0,...,32465.0,455.0,1957.0,6480.0,14411.0,2386.0,8227.0,NaN,0,0


In [350]:
dataframe_test = dataframe[dataframe['year'] == 2020].reset_index().drop(['index'], axis=1)
dataframe_train = dataframe[(dataframe['year'] == 2014) | (dataframe['year'] == 2008)
                           ].dropna(subset = ['incumbent', 'total_unemployed', 'asyl', 'cost_o_liv', 'capital_crimes'
                                             ]).reset_index().drop(['index'], axis=1)
len(dataframe_train)

9091

In [355]:
X_names = ['female_total_x', 'cost_o_liv', 'asyl', 'BP_votes', 'CSU_votes', 'SPD_votes', 
           'Linke_votes', 'FDP_votes', 'Grüne_votes', 'AfD', 'BP', 'CSU', 'FDP', 
           'FW', 'Grüne', 'Linke', 'SPD', 'age0_5', 'age15_18', 'age18_24', 'age25_30', 'age30_40', 
           'age40_50', 'age50_65', 'age65plus', 'age6_14', 'anteil_einkommensteuer', 'anteil_umsatzsteuer',
           'capital_crimes', 'doctor', 'gewerbesteuerumlage', 'importance_climate', 'importance_social_sec', 'incumbent',
           'istaufkommen', 'mayor', 'muslim', 'new_list',
           'non_muslim_migrant', 'place_list', 'globalisation',
           'politician', 'royal', 'search_result_stan', 'steuern_total', 'students',
           'total_foreigner', 'total_pop', 'total_unemployed', 'unemployed_longtime', 'youth_list', 'ödp']

to_scale = ['female_total_x', 'cost_o_liv', 'asyl',  'BP_votes', 'CSU_votes', 'SPD_votes', 'Linke_votes', 'FDP_votes',
            'gewerbesteuerumlage', 'Grüne_votes', 'age0_5', 'age15_18', 'age18_24', 'age25_30', 'age30_40',
            'age40_50', 'age50_65', 'age65plus', 'age6_14', 'capital_crimes',
            'istaufkommen', 'students', 'total_foreigner', 'total_pop', 'total_unemployed', 'unemployed_longtime', 
            'steuern_total', 'anteil_einkommensteuer', 'anteil_umsatzsteuer']

y_train = dataframe_train['votes']

X_train = dataframe_train[X_names]
X_test = dataframe_test[X_names]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled_2 = StandardScaler().fit_transform(X_train[to_scale])
X_test_scaled_2 = StandardScaler().fit(X_train[to_scale]).transform(dataframe_test[to_scale])

X_train_scaled[to_scale] = X_train_scaled_2
X_test_scaled[to_scale] = X_test_scaled_2

In [352]:
X_test_scaled.columns[X_test_scaled.isna().any()].tolist()

[]

In [353]:
X_train_scaled.columns[X_test_scaled.isna().any()].tolist()

[]

In [378]:
param_test1 = {'n_estimators':range(20,200,20)}
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,100)}
param_test3 = {'min_samples_leaf': range(10, 200, 30) }

t1 = time.time()
gradient_boosting_cv = GridSearchCV(estimator = GradientBoostingRegressor
                                    (random_state=1, min_samples_leaf = 40, 
                                    n_estimators=180, loss = 'ls', learning_rate = 0.1, max_features = 'sqrt',
                                    criterion = 'mse', verbose = 0, max_depth = 15, min_samples_split = 700)
                                    , param_grid = param_test3, cv=10, error_score='raise', verbose = 1, n_jobs = -1)
t_grad_cv = time.time() - t1

t2 = time.time()
gradient_boosting_cv.fit(X_train_scaled, y_train)
t_grad_reg = time.time() - t2

gradient_boosting_cv.best_params_, gradient_boosting_cv.best_score_, t1, t2, gradient_boosting_cv.score(X_train_scaled,y_train)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   26.6s finished


({'min_samples_leaf': 40},
 0.39278534296541345,
 1582890909.0109415,
 1582890909.0139034,
 0.9830829450247932)

In [356]:
grad_boost = GradientBoostingRegressor(random_state=1, min_samples_leaf = 40, 
                                    n_estimators=180, loss = 'ls', learning_rate = 0.1, max_features = 'sqrt',
                                    criterion = 'mse', verbose = 0, max_depth = 15, min_samples_split = 700)

grad_boost.fit(X_train_scaled, y_train)
grad_boost.score(X_train_scaled, y_train), grad_boost.feature_importances_

(0.9830829450247932,
 array([2.88102014e-02, 1.36122490e-02, 5.97338106e-02, 3.39416964e-02,
        6.64293288e-02, 1.03994296e-02, 9.45194931e-03, 2.81235107e-02,
        8.06610064e-02, 5.25044947e-05, 1.66205792e-02, 1.67249910e-01,
        2.63345763e-03, 7.06970494e-04, 9.83158641e-03, 5.37244437e-05,
        1.00586741e-01, 8.71081717e-03, 2.41539927e-03, 2.67500432e-02,
        2.23490457e-02, 1.41777471e-02, 3.90451751e-02, 8.44320865e-03,
        9.16862074e-03, 2.96663100e-03, 2.34525308e-02, 3.51036896e-02,
        1.72370565e-02, 1.40800270e-04, 3.40539153e-02, 0.00000000e+00,
        0.00000000e+00, 3.27450411e-02, 2.27617936e-03, 0.00000000e+00,
        1.43113424e-06, 1.97361035e-04, 1.60629227e-05, 1.08830592e-02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.41897942e-04,
        2.67174941e-02, 3.68825178e-03, 3.52865326e-03, 2.42246160e-02,
        1.12263229e-02, 1.06946947e-02, 4.92050419e-06, 1.40677401e-04]))

In [379]:
param_grid_1 = {
    #'max_depth': range(10,100,10),
    #'min_samples_leaf': [4, 8, 12, 16, 20],
    'min_samples_split': [10,20,30,40,50],
    'n_estimators': range(10,100,10),
}

random_forest = GridSearchCV(estimator = RandomForestRegressor()
                             , param_grid = param_grid_1, cv = 10, verbose = 1, n_jobs = -1)
t2 = time.time()
random_forest.fit(X_train_scaled, y_train)
t_random_forest = time.time() - t2


random_forest.score(X_train_scaled,y_train), random_forest.best_params_

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  5.8min finished


(0.9914854080498726, {'min_samples_split': 30, 'n_estimators': 50})

In [380]:
rf = RandomForestRegressor(max_depth= 60, min_samples_leaf= 4, min_samples_split= 30, n_estimators = 50)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=60,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=30,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [375]:
Cs = [1, 2, 5, 8, 10, 20]
param_grid_svr = {'C': Cs, 'max_iter': range(1000, 10000, 1000)}

svr_cv = GridSearchCV(estimator = LinearSVR(), param_grid = param_grid_svr, cv=10, n_jobs = -1, verbose = 1)
svr_cv.fit(X_train_scaled, y_train)
svr_cv.best_params_, svr_cv.score(X_train_scaled, y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 207 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 457 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.8min finished
C:\Users\mariu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'C': 1, 'max_iter': 7000}, -162.774691758675)

In [358]:
svr = LinearSVR(C=1, max_iter = 7000)
svr.fit(X_train_scaled, y_train)

LinearSVR(C=1, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=5000,
          random_state=None, tol=0.0001, verbose=0)

In [381]:
ensemble_regression = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr)])

param_ensemble = {'weights': [(2,1,1),(2,2,1),(2,1,2),(1,2,2),(1,1,2),(1,2,1),
                             (1,1,1),(1,0,2),(2,0,1),(0,1,2),(0,2,1),(2,1,0),(1,2,0),
                              (3,3,2),(3,2,3),(2,3,3),(2,2,3),(2,3,2),(3,2,2),(3,2,1),(3,1,2),(1,3,2),(2,3,1),(2,1,3),
                              (1,2,3), (3,1,1),(1,3,1),(1,1,3),(3,2,0),(3,0,2),(0,3,2),(2,3,0),(0,2,3),(2,0,3),
                              (3,1,0),(3,0,1),(1,3,0),(0,3,1),(0,1,3),(1,3,0)
                             ]}
ensemble_cv = GridSearchCV(estimator = ensemble_regression, param_grid = param_ensemble , cv=10, n_jobs = 4, verbose = 2)
ensemble_cv.fit(X_train_scaled, y_train)

ensemble_cv.best_params_, ensemble_cv.score(X_train_scaled, y_train)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   45.9s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  9.2min
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed: 10.1min finished
C:\Users\mariu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'weights': (3, 1, 0)}, 0.9904758754899454)

In [382]:
voting_reg = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr)], weights = (3,1,0))
voting_reg.fit(X_train_scaled, y_train)

VotingRegressor(estimators=[('random_forest',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=60,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=4,
                                                   min_samples_split=30,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=50, n_jobs=None,
                                                   oob_score=False,
                                                   random_state=None, verbose=0,
 

In [269]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score


def display_score(reg, mse_reg, r2):
    
    """ Insert explanation """
    
    reg_rmse = np.sqrt(-mse_reg)
    
    print('MSE                                  ')
    print('Scores:', reg_rmse, reg)
    print('Mean:', reg_rmse.mean(), reg)
    print('standard Deviation:', reg_rmse.std(), reg)
    print('R2                                   ')
    print('Scores:', r2, reg)
    print('Mean:', r2.mean(), reg)
    print('standard Deviation:', r2.std(), reg)

In [383]:
""" Gradient Boosting"""

mse_gb_train = cross_val_score(grad_boost, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_gb_train = cross_val_score(grad_boost, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("Gradient Boosting", mse_gb_train, r2_gb_train)

MSE                                  
Scores: [32345.47976408 32760.59915955  1323.57890206  1678.36898552
  1587.84273238  1339.54993649 12519.73702012  7775.66956973
  3750.1725834   2113.87499881] Gradient Boosting
Mean: 9719.487365215142 Gradient Boosting
standard Deviation: 11913.958655286326 Gradient Boosting
R2                                   
Scores: [ 0.17446902  0.40100545  0.41550198  0.32931044  0.57276722  0.62812933
  0.59793761 -0.68760007  0.74877784  0.74779479] Gradient Boosting
Mean: 0.3928093601631192 Gradient Boosting
standard Deviation: 0.400267088226607 Gradient Boosting


In [384]:
mse_rf_train = cross_val_score(rf, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_rf_train = cross_val_score(rf, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("RF", mse_rf_train, r2_rf_train)

MSE                                  
Scores: [29789.1387013  30597.10385902  1036.0257341   1179.91791612
  1285.47486777  1689.69791983 15589.42037887  4608.48141658
  6299.38560138  1636.48680986] RF
Mean: 9371.113320483139 RF
standard Deviation: 11220.514504818224 RF
R2                                   
Scores: [0.48969702 0.46773412 0.66618032 0.6652437  0.71402199 0.46237396
 0.37431025 0.40821733 0.32074952 0.84907856] RF
Mean: 0.5417606775417128 RF
standard Deviation: 0.16249884644128773 RF


In [385]:
mse_rf_train = cross_val_score(svr, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_rf_train = cross_val_score(svr, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("SVR", mse_rf_train, r2_rf_train)

MSE                                  
Scores: [ 34875.78177371  43111.83316813 140159.02975177   1760.85943797
   2061.29887712   1867.72279096  21058.40393514   5537.90623589
   7717.73443325   3799.13925351] SVR
Mean: 26194.970965744524 SVR
standard Deviation: 40517.52525575574 SVR
R2                                   
Scores: [ 4.01599680e-02 -3.66355155e-02 -7.59917704e+03  2.61900955e-01
  2.80263716e-01  2.77942558e-01 -1.38274014e-01  1.43506013e-01
 -6.52122560e-02  1.83380891e-01] SVR
Mean: -759.8230012183772 SVR
standard Deviation: 2279.784685595608 SVR


In [386]:
mse_rf_train = cross_val_score(voting_reg, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_rf_train = cross_val_score(voting_reg, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("Ensemble", mse_rf_train, r2_rf_train)

# -> Voting regression with weight combination 3,2,1 appears to be the most credible estimator
# Let's check out the redictions

MSE                                  
Scores: [26163.17186258 31120.91404656  1047.43432349  1213.67503454
  1303.84822922  1291.72869611 14815.13310429  4656.21409358
  5768.47542782  1415.91635731] Ensemble
Mean: 8879.651117550844 Ensemble
standard Deviation: 10709.473137210174 Ensemble
R2                                   
Scores: [0.28481529 0.45888252 0.65729293 0.65601665 0.70413597 0.41275058
 0.43158296 0.19574248 0.41683233 0.88098995] Ensemble
Mean: 0.5099041666041908 Ensemble
standard Deviation: 0.19867162119062726 Ensemble


In [387]:
rf_pred = rf.predict(X_test_scaled)

rf_pred = pd.DataFrame({'Prediction': ensemble_regression_pred, 'Votes': dataframe_test['votes'], 'city': dataframe_test['city'],
                                      'party': dataframe_test['party'], 'last_name': dataframe_test['last_name'], 'city_id': dataframe_test['city_id'],
                                     'county_id': dataframe_test['county_id'], 'first_name': dataframe_test['first_name'],
                                      'place_list': dataframe_test['place_list']})

rf_pred.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\forecast_candidates.xlsx')

In [389]:
list_coefficients = pd.DataFrame({'Variable': X_train.columns, 'Coefficients RF':rf.feature_importances_
                                 })
list_coefficients

,Variable,Coefficients RF
0,female_total_x,3.191853e-02
1,cost_o_liv,7.180290e-03
2,asyl,3.579835e-02
3,BP_votes,8.036199e-03
4,CSU_votes,3.982262e-02
5,SPD_votes,3.574540e-02
6,Linke_votes,5.588868e-02
7,FDP_votes,4.221969e-02
8,Grüne_votes,3.486569e-02
9,AfD,6.582809e-05
